In [170]:
import json
import pandas as pd
import numpy as np

In [171]:
f = open('trainings.txt')
data = json.load(f)

In [172]:
# Checking whether data frame and normal count is of same size
count = 0
for i in data:
    count+= len(i['completions'])
print(len(data), count)

1000 2967


In [173]:
df_data = []
for record in data:
    for sub_record in record['completions']:
        df_data.append({
            'name_student': record['name'],
            'name_course': sub_record['name'],
            'timestamp': sub_record['timestamp'],
            'expires': sub_record['expires']
        })
df_data = pd.DataFrame(df_data)
df_data['timestamp'] = pd.to_datetime(df_data['timestamp'], format = '%m/%d/%Y')
df_data['expires'] = pd.to_datetime(df_data['expires'], format = '%m/%d/%Y')

df_data

,name_student,name_course,timestamp,expires
0,Jaelyn Quinn,Electrical Safety for Labs,2022-08-31,NaT
1,Jaelyn Quinn,Safe Handling of Human Cell Lines/Materials in...,2023-10-30,NaT
2,Jaelyn Quinn,Awareness Training for the Transport of Hazard...,2023-10-05,NaT
3,Jaelyn Quinn,X-Ray Safety,2023-04-08,NaT
4,Asia Duke,X-Ray Safety,2022-09-01,NaT
...,...,...,...,...
2962,Jadon Harrell,Physical Science Responsible Conduct of Resear...,2023-06-10,NaT
2963,Jadon Harrell,Animal Care And Use Risk Assessment,2023-08-22,2024-08-21
2964,Jadon Harrell,OHS Training,2022-03-14,2023-03-14
2965,Jadon Harrell,Occupational Exposure to Bloodborne Pathogens,2022-11-03,2023-11-03


In [174]:
# Removing duplicates of training and placing only the training that is most recent.
df_data = df_data.sort_values('timestamp', ascending=False).drop_duplicates(['name_student', 'name_course'])

In [175]:
# Task 1
# List each completed training with a count of how many people have completed that training.
count_courses = df_data.groupby('name_course')['name_student'].nunique()
count_courses.to_json('completed_training_count1.json', indent=4)
count_courses.to_json()

'{"Animal Care And Use Risk Assessment":99,"Awareness Training for the Transport of Hazardous Material":110,"Basic Training Program for Animal Users":113,"Chemical Waste Requirements":111,"DOT Hazard Material Awareness":116,"Electrical Safety for Labs":118,"Federally Required RCR Training - Must be delivered by PI (Not available online)":114,"Health Screening Questionnaire":114,"IRB Quiz":99,"Laboratory Safety Training":113,"Medical Records":104,"NIH Guidelines Overview":94,"OHS Training":116,"Occupational Exposure to Bloodborne Pathogens":102,"Physical Science Responsible Conduct of Research Course 1.":111,"Radiation Safety Annual Refresher":121,"Radioactive Materials Safety Training":121,"Retraining in working with the IACUC":114,"Safe Handling of Human Cell Lines\\/Materials in a Research Laboratory":99,"Safety Practices and Procedures to Prevent Zoonotic Diseases While Working With Cattle":114,"Transportation of Infectious Substances, Category B":107,"Understanding Biosafety":123,"

In [176]:
# Task 2
# Listing all people that completed that training in the specified fiscal year (2024).
# With Trainings = "Electrical Safety for Labs", "X-Ray Safety", "Laboratory Safety Training

def fiscal_year(df,trainings,fiscal_year):
    fiscal_year_start = pd.Timestamp(year=fiscal_year - 1, month=7, day=1)
    fiscal_year_end = pd.Timestamp(year=fiscal_year, month=6, day=30)

    # Filter the DataFrame
    filtered_df = df[(df['name_course'].isin(trainings)) & 
                     (df['timestamp'] >= fiscal_year_start) & 
                     (df['timestamp'] <= fiscal_year_end)]

    # Group by 'name_course' and list all 'name_student'
    result = filtered_df.groupby('name_course')['name_student'].apply(list)
    result.to_json('training_list1.json', indent=4)
   
    return result.to_json()

In [177]:
trainings_list = ["Electrical Safety for Labs", "X-Ray Safety", "Laboratory Safety Training"]
fiscal_year(df_data,trainings_list, 2024)

'{"Electrical Safety for Labs":["Teresa Esparza","Leandro Krause","Desiree Potts","Zavier Maldonado","Calvin Rubio","Brogan Stein","Antwan Villanueva","Lexie Ho","Sandra Manning","Gerald Parsons","Jamison Lawson","Humberto Huerta","Reed Ramos","Allie Barnes","Kierra Grimes","Martin Dean","Armani Tapia","Audrina Eaton","Charity Lucas"],"Laboratory Safety Training":["Travis Grant","Makenzie Pollard","Skye Valenzuela","Lawson Quinn","Annabella Jacobson","Karli Craig","Ariella David","Dawson Compton","Conner Warner","Nayeli Mckenzie","Vaughn Mcmillan","Zane Mcgrath","Dania Parrish","Matias Oconnell","Roy Robbins","Noah Burch","Eden Nash","Kaitlyn Orozco","Niko Frey","Sasha Armstrong"],"X-Ray Safety":["Hadassah Hampton","Anabelle Braun","Ann Knight","Blaze Joyce","Gloria Mccarty","Mitchell Ware","Antwan Villanueva","Jagger Hoffman","Rayan Valencia","Noemi Cisneros","Soren Chaney","Cloe Williamson","Nayeli Ortiz","Ezra Powers","Ainsley Drake","Destinee Odonnell","Irvin Estrada","Brynn Marks"

In [178]:
# Task 3
# Find all people that have any completed trainings that have already expired, or will expire within one month
def find_expiring_trainings(df, user_date):
    # Convert 'expires' to datetime if it's not already
    df['expires'] = pd.to_datetime(df['expires'])

    # Convert user_date to datetime
    specified_date = pd.to_datetime(user_date)

    # Calculate expiration status
    df['status'] = df['expires'].apply(
        lambda x: 'expired' if x <= specified_date else (
            'expires soon' if x <= specified_date + pd.DateOffset(months=1) else 'valid'
        )
    )

    # Filter to include only expired or expires soon
    df_filtered = df[df['status'].isin(['expired', 'expires soon'])]

    # Group by 'name_student' and list all 'name_course' and their 'status'
    result = df_filtered.groupby('name_student').apply(
        lambda x: x[['name_course', 'status']].to_dict('records')
    )
    result.to_json('training_expiry_list1.json', indent=4)
    return result.to_json()

In [179]:
find_expiring_trainings(df_data, '10/1/2023')

'{"Abraham Mendoza":[{"name_course":"Medical Records","status":"expires soon"}],"Adalynn Dunlap":[{"name_course":"Transportation of Infectious Substances, Category B","status":"expired"}],"Adan Lowery":[{"name_course":"Medical Records","status":"expired"}],"Adan Salazar":[{"name_course":"Radioactive Materials Safety Training","status":"expired"}],"Adison Brewer":[{"name_course":"Retraining in working with the IACUC","status":"expired"}],"Aditya Alvarez":[{"name_course":"Occupational Exposure to Bloodborne Pathogens","status":"expired"}],"Adriana Figueroa":[{"name_course":"Safety Practices and Procedures to Prevent Zoonotic Diseases While Working With Cattle","status":"expired"},{"name_course":"Radioactive Materials Safety Training","status":"expired"}],"Adrienne Yates":[{"name_course":"Transportation of Infectious Substances, Category B","status":"expired"}],"Aiyana Stone":[{"name_course":"Health Screening Questionnaire","status":"expired"}],"Alannah Booth":[{"name_course":"Using Hazar